# AFAP Ratio Engine Validation

This notebook validates:
1. Deterministic ratio calculations from `ratio_engine_core`
2. Threshold-based interpretation from `ratio_engine_eval`
3. Schema integrity and orchestration readiness

This notebook is part of AFAP Phase 3 (Locked Engines).


# Project Root Path Setup

This cell ensures the notebook can import modules from the project, regardless of where the notebook is located.

- `os.getcwd()` gets the current working directory of the notebook.
- We append `".."` to move to the project root.
- `sys.path.insert(0, PROJECT_ROOT)` temporarily adds it to Python’s module search path.
- This allows imports like `from engines.ratio_engine_core import ratio_engine` to work.

This step is crucial when working with Jupyter notebooks in a subfolder.


In [1]:
import sys
import os

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print("Project root added:", PROJECT_ROOT)


Project root added: c:\Users\ADMIN\Documents\My Documents\MyDataAnalysis\Financial statement analysis\financial-analysis-pipeline


# Import Modules & Configure Display

- Standard Python modules (`sys`, `os`) for path handling.
- `pandas` and `numpy` for data handling and numerical operations.
- Import AFAP engines: `ratio_engine`, `evaluate_ratios`, `validate_engine_output`.
- `pd.set_option` ensures floating-point numbers display consistently (4 decimal places), making ratio comparisons easier.


In [6]:
import pandas as pd
import numpy as np
import sys
import os

from engines.ratio_engine_core import ratio_engine
from engines.ratio_engine_eval import evaluate_ratios
from engines.schema_validator import validate_engine_output
from engines.ratio_validator import validate_ratios

In [3]:
pd.set_option("display.float_format", "{:.4f}".format)


# Load Cleaned Financial Statements

- Reads `financial_statements.csv` containing raw financial data (Company, Year, FS Category, FS Subcategory, Amount).
- `head()` allows quick inspection of the data.
- This is the input to `ratio_engine_core`.


In [4]:
financials_df = pd.read_csv("../data/cleaned/Kenya_Airways.csv")

financials_df.head()


,Company,Year,FS Category,FS Subcategory,Amount
0,Kenya Airways,2021,Assets,Current Assets,"25,685"
1,Kenya Airways,2021,Assets,Non-Current Assets,"129,870"
2,Kenya Airways,2021,Assets,Inventory,"2,152"
3,Kenya Airways,2021,Liabilities,Current Liabilities,"80,965"
4,Kenya Airways,2021,Liabilities,Non-Current Liabilities,"157,927"


# Check Column Names and Data Types

- `info()` verifies column types, non-null counts, and ensures all required columns exist.
- Catch missing or misnamed columns before passing to ratio engine.


In [5]:
financials_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Company         44 non-null     object
 1   Year            44 non-null     int64 
 2   FS Category     44 non-null     object
 3   FS Subcategory  44 non-null     object
 4   Amount          44 non-null     object
dtypes: int64(1), object(4)
memory usage: 1.8+ KB


# Compute Ratios Using ratio_engine_core

- Runs deterministic ratio calculations for each company-year combination.
- Returns a list of dictionaries, each containing:
  - Engine name
  - Company and Year
  - `metrics` dictionary with calculated ratios
  - `flags`, `severity`, `explanation` fields (currently empty or default)


In [7]:
ratio_results = ratio_engine(financials_df)

ratio_results


NameError: name 'validate_ratios' is not defined

In [7]:
financials_df.dtypes


Company           object
Year               int64
FS Category       object
FS Subcategory    object
Amount            object
dtype: object

# Flatten ratio_engine output

- `pd.json_normalize` converts list-of-dicts output to a pandas DataFrame.
- Metrics are nested dicts (`metrics`) and initially appear as `metrics.<ratio_name>` columns.
- `head()` gives a preview of the flattened ratio values for inspection.


In [8]:
ratio_df = pd.json_normalize(ratio_results)

ratio_df.head()


,engine,Company,Year,severity,explanation,metrics.current_ratio,metrics.quick_ratio,metrics.gross_margin,metrics.operating_margin,metrics.net_margin,metrics.debt_equity,metrics.interest_coverage,metrics.asset_turnover,metrics.roa,metrics.roe
0,ratio_engine,Kenya Airways,2021,stable,Canonical financial ratios,0.3172,0.2907,-0.0969,-1.1938,-1.3253,-2.8666,-8.9301,0.4514,-0.5983,1.1167
1,ratio_engine,Kenya Airways,2022,stable,Canonical financial ratios,0.3553,0.3359,-0.0481,-1.0962,-1.3774,-2.5627,-3.8920,0.6914,-0.9523,1.4882
2,ratio_engine,Kenya Airways,2023,stable,Canonical financial ratios,0.4190,0.3965,0.0578,-0.8844,-1.0715,-2.3023,-4.7042,1.0107,-1.0829,1.3853
3,ratio_engine,Kenya Airways,2024,stable,Canonical financial ratios,0.3483,0.3230,0.3232,0.1965,0.1368,-2.5146,3.3187,1.0524,0.1440,-0.2181
